In [32]:
import pandas as pd
import numpy as np
import os
import janitor

In [2]:
# load files
files = os.listdir("../Datasets/")

In [122]:
class Processing:

    ARCHIVOSORIGINALES = ['BD tesis DEPURADA ESP 2013-2018.xlsx',
    'Resultados descentralizadas pequeñas_2018.xlsx',
    'ResultadosFinalesFURAGV2019.xlsx',
    'ResultadosFURAG_2018.xlsx',
    'ResultadosMDI_vig2020.xlsx',
    'Resultados_Desempeno_v2021.xlsx']

    SECCIONES = ["balance","activo_corriente","activo_no_corriente","pasivo_corriente","pasivo_no_corriente",
    "estado_resultados"]

    COLUMNAS_SECCIONES = [7,12,12,16,16,11]
    for indice,col in enumerate(COLUMNAS_SECCIONES):
        if indice!=0:
            COLUMNAS_SECCIONES[indice] = COLUMNAS_SECCIONES[indice-1]+COLUMNAS_SECCIONES[indice]

    def __init__(self,file_list):
        self.file_list = file_list
    
    def read_files(self):
        df_list = list()

        for file in self.file_list:
            df = pd.ExcelFile("../Datasets/"+file)
            if (len(df.sheet_names) > 1) & (file==Processing.ARCHIVOSORIGINALES[0]):
                final_df = pd.DataFrame()
                # final_df = list()
                for sheets in df.sheet_names:

                    sheet_df = df.parse(sheets)
                    final_colnames = sheet_df.iloc[0]
                    final_colnames.replace(np.nan,"Empresa",inplace=True)                     
                    sheet_df.columns = final_colnames
                    sheet_df.drop(sheet_df.index[0],inplace=True)
                    sheet_df["year"] = sheets
                    sheet_df.set_index(sheet_df["year"]+sheet_df.index.astype(str),inplace=True)

                    if sheets != "2013":
                        # add a new column
                        sheet_df = janitor.clean_names(sheet_df)
                    
                    elif sheets =="2013":
                        sheet_df.rename(columns={"Otros Activos Pesos":"Otros Pasivos Pesos"},inplace=True)
                        sheet_df = janitor.clean_names(sheet_df)

                    lista_columnas = sheet_df.columns.tolist()

                    for indice,columna in enumerate(lista_columnas):
                        
                        if indice<=Processing.COLUMNAS_SECCIONES[0]:
                            lista_columnas[indice] = Processing.SECCIONES[0]+"_"+columna
                        
                        elif indice<=Processing.COLUMNAS_SECCIONES[1]:
                            lista_columnas[indice] = Processing.SECCIONES[1]+"_"+columna
                        
                        elif indice<=Processing.COLUMNAS_SECCIONES[2]:
                            lista_columnas[indice] = Processing.SECCIONES[2]+"_"+columna

                        elif indice<=Processing.COLUMNAS_SECCIONES[3]:
                            lista_columnas[indice] = Processing.SECCIONES[3]+"_"+columna

                        elif indice<=Processing.COLUMNAS_SECCIONES[4]:
                            lista_columnas[indice] = Processing.SECCIONES[4]+"_"+columna

                        elif indice<=Processing.COLUMNAS_SECCIONES[5]:
                            lista_columnas[indice] = Processing.SECCIONES[5]+"_"+columna
                    
                    sheet_df.set_axis(lista_columnas,axis=1,inplace=True)
                    final_df = pd.concat([final_df,sheet_df],axis=0)
                
            else:
                final_df = df.parse(0)  
                              
            df_list.append(final_df)

        self.df_list = df_list

In [139]:
datasets_processing = Processing(files)
datasets_processing.read_files()

In [264]:
datasets_processing.df_list[0][1].sort_index(axis=1,inplace=True)
datasets_processing.df_list[0][0].sort_index(axis=1,inplace=True)

In [140]:
test = datasets_processing.df_list[3].copy()
test.columns = test.iloc[3]
test.drop(test.index[:4],inplace=True)
test.clean_names()

3,codigosigep,entidad,tipo_de_informe,orden,region_segun_el_pnd_,id_departamento,departamento,id_municipio,municipio,categoria_municipal,...,politica_7_seguridad_digital,politica_8_defensa_juridica,politica_9_transparencia_acceso_a_la_informacion_y_lucha_contra_la_corrupcion,politica_10_servicio_al_ciudadano,politica_11_racionalizacion_de_tramites,politica_12_participacion_ciudadana_en_la_gestion_publica,politica_13_seguimiento_y_evaluacion_del_desempeno_institucional,politica_14_gestion_documental,politica_15_gestion_del_conocimiento,politica_16_control_interno
4,5291,CAJA PROMOTORA DE VIVIENDA MILITAR Y DE POLICÍA,MIPG,NACIONAL,CENTRAL,11,"Bogotá, D. C.",11001,"Bogotá, D. C.",ESP,...,32.483138,37.879235,42.23878,38.899548,29.939143,38.549487,27.673407,39.218949,24.679524,35.083396
5,0021,DEPARTAMENTO NACIONAL DE PLANEACIÓN,MIPG,NACIONAL,CENTRAL,11,"Bogotá, D. C.",11001,"Bogotá, D. C.",ESP,...,32.483138,37.879235,42.23878,38.899548,29.939143,38.549487,27.673407,39.218949,24.679524,35.083396
6,0030,SUPERINTENDENCIA DE SOCIEDADES,MIPG,NACIONAL,CENTRAL,11,"Bogotá, D. C.",11001,"Bogotá, D. C.",ESP,...,32.483138,37.879235,42.23878,38.899548,29.939143,38.549487,27.673407,39.218949,24.679524,35.083396
7,0011,MINISTERIO DE EDUCACIÓN NACIONAL,MIPG,NACIONAL,CENTRAL,11,"Bogotá, D. C.",11001,"Bogotá, D. C.",ESP,...,32.483138,37.879235,42.23878,38.899548,29.939143,38.549487,27.673407,39.218949,24.679524,35.083396
8,5324,FONDO DE GARANTÍAS DE INSTITUCIONES FINANCIERAS,MIPG,NACIONAL,CENTRAL,11,"Bogotá, D. C.",11001,"Bogotá, D. C.",ESP,...,32.483138,37.879235,42.23878,38.899548,29.939143,38.549487,27.673407,39.218949,24.679524,35.083396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,1528,CORPORACIÓN PARA EL DESARROLLO SOSTENIBLE DE L...,MECI,NACIONAL,CARIBE,70,Sucre,70708,San Marcos,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.087125
215,0895,CORPORACION AUTONOMA REGIONAL DEL RIO GRANDE D...,MECI,NACIONAL,SANTANDERES,68,Santander,68081,Barrancabermeja,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.087125
216,0883,CORPORACIÓN AUTÓNOMA REGIONAL DEL SUCRE - CARS...,MECI,NACIONAL,CARIBE,70,Sucre,70001,Sincelejo,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.087125
217,0893,CORPORACIÓN PARA EL DESARROLLO SOSTENIBLE DEL ...,MECI,NACIONAL,SEAFLOWER,88,Archipiélago De San Andrés Y Providencia,88564,Providencia,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.087125
